In [1]:
from dataclasses import dataclass
from datetime import date
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd

In [47]:
from dataclasses import dataclass, field
from datetime import date


@dataclass
class Bond:
    """A model for a bond in farabourse"""

    id: int = field(default=None)
    sym: str = field(default=None)
    name: str = field(default=None)
    fv: float = field(default=0)  # face value
    cap: float = field(default=0)  # Issued Marketcap
    pcap: float = field(default=0)
    issue_date: date = field(default=None)
    period: int = field(default=0)  # number of months
    desc1: str = field(default=None)
    desc2: str = field(default=None)
    nom_rate: float = field(default=0)  # nominal rate
    count: int = field(default=0)
    pcount: int = field(default=0)
    mat_date: date = field(default=None)
    copoun: int = field(default=0)
    fbprice: float = field(default=0)  # Spot price in farabourse
    fblastdate: date = field(default=None)  # date of the last trade
    fbytm: float = field(default=0)  # YTM calculated in Farabourse Website
    fbret: float = field(default=0)  # Simple Return Calculated by Farabourse
    ytm: float = field(default=0)

    def _calculate_ytm(self):
        ytm = 2
        return ytm

    def __repr__(self):
        return f"""Bond(
    id={self.id},
    sym={self.sym},
    name={self.name},
    fv={self.fv},
    cap={self.cap},
    pcap={self.pcap},
    issue_date={self.issue_date},
    period={self.period},
    desc1={self.desc1},
    desc2={self.desc2},
    nom_rate={self.nom_rate},
    count={self.count},
    pcount={self.pcount},
    mat_date={self.mat_date},
    copoun={self.copoun},
    fbprice={self.fbprice},
    fblastdate={self.fblastdate},
    fbytm={self.fbytm},
    fbret={self.fbret},
    ytm={self.ytm}
)"""


In [52]:
def rename_and_reorder_columns(df):
    """Renames and reorders the columns of the given DataFrame."""
    
    # Define a dictionary to map original column names to new column names
    column_name_mapping = {
        "نماد": "sym",
        "قیمت معامله شده هر ورقه": "fbprice",
        "تاریخ آخرین روز معاملاتی": "fblastdate",
        "تاریخ سررسید": "mat_date",
        "YTM": "fbytm",
        "بازده ساده": "fbret",
        "id": "id"
    }
    
    # Rename the columns in the DataFrame
    df = df.rename(column_name_mapping, axis=1)
    
    # Reorder the columns in the DataFrame based on the new column names
    df = df[list(column_name_mapping.values())]

    return df


In [53]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd

def get_bonds_list():
    """Fetches bond data from the website and returns a DataFrame with the data."""
    
    # Initialize an empty list of bonds and the base URL
    bonds = []
    BASE_URL = "https://www.ifb.ir/ytm.aspx"

    # Make an HTTP request to the base URL
    req = Request(BASE_URL, headers={"User-Agent": "Mozilla/5.0"})
    html_page = urlopen(req).read()

    # Parse the HTML content
    soup = BeautifulSoup(html_page, "html.parser")

    # Initialize an empty DataFrame to store bond data
    df = pd.DataFrame()

    # Define bond classes and find all table elements in the HTML
    bonds_classes = ['KhazanehGrid', 'mGrid']
    tables = soup.find_all("table")
    
    # Filter tables based on bond classes
    tables = [table for table in tables if set(table["class"]) & set(bonds_classes)]

    # Iterate through the filtered tables and extract bond data
    for table in tables:
        # Convert the HTML table to a DataFrame
        new_df = pd.read_html(str(table))[0]

        # Extract bond IDs from the table
        hrefs = [(x['href'], x["symid"]) for x in table.find_all("a")]
        ids = [x[-1] for x in hrefs]

        # Check if the number of IDs matches the number of rows in the DataFrame
        assert len(ids) == new_df.shape[0]
        
        # Add the IDs to the new DataFrame and concatenate it with the main DataFrame
        new_df["id"] = ids
        df = pd.concat([df, new_df])

    return rename_and_reorder_columns(df)


In [59]:
df = get_bonds_list()
bonds = [Bond(**x) for x in df.to_dict("records")]